# Summery: Featuring engineering + LGBM
This notebook is a modified from notebook provided by YE_AI [link](https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-cv-0-799-lb-0-799?scriptVersionId=171264491). If you like my notebook remember to like YE_AI notebook also. 

***

### Modification Done:
1. Add sum, kurtosis, Quartile 1 and Quartile 3 to paragraph and sentence features.
2. Add spelling mistake counting and Extra text processing.
3. Add TFIDF vector.
4. Add TFIDF word. 

***

### Score:
| Setup | Validation score | LB score |
| :--- | :--- | :-- |
| LGBM <br>+ feature engineering (Paragraph, sentence & word based features) | 0.7351538 | 0.780 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) | 0.7428280 | 0.782 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Extra text processing | 0.7493708 | 0.790 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Extra text processing <br>+ Character TFIDF feature (`whole text is taken for TFIDF`) | 0.8022457 | 0.797 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Extra text processing <br>+ Character TFIDF feature (`whole text is taken for TFIDF`)<br>+ Word TFIDF feature (`whole text is taken for TFIDF`) | 0.8019827 | 0.797 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Extra text processing <br>+ Character TFIDF feature (`whole text is taken for TFIDF`)<br>+ Word TFIDF feature (`whole text is taken for TFIDF`) <br>+ calc a and b values based on Fold | 0.8026114 | 0.796 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Extra text processing <br>+ Character TFIDF feature (`whole text is taken for TFIDF`)<br>+ Update Word TFIDF feature (min_df=0.15 & max_df=0.85) (`whole text is taken for TFIDF`) <br>+ calc a and b values based on Fold | 0.7996054 | 0.799 |
| LGBM <br>+ feature engineering (Paragraph, sentence, word based features & spelling mistake counts) <br>+ Update Extra text processing [**[Link](#extra-feature)**] <br>+ Character TFIDF feature (`whole text is taken for TFIDF`)<br>+ Update Word TFIDF feature (min_df=0.15 & max_df=0.85) (`whole text is taken for TFIDF`) <br>+ calc a and b values based on Fold | 0.8006839 | 0.802 |

**Extra text processing**:
> * Contraction Expension e.g. I'll --> i will, this is added as a text processing step.
> * Punctuation removal is applied:
    - When extra features are generation [**[Link](#extra-feature)**]
> 
> **Note**: The <u>Extra text processing</u> is taken from the notebook [here](https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817)

***

### References:
* Paragraph, sentence & word based features [source](https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-cv-0-799-lb-0-799).
* Spelling mistake counting [here](https://www.kaggle.com/code/tsunotsuno/updated-debertav3-lgbm-with-spell-autocorrect).
* Extra text processing [here](https://www.kaggle.com/code/xianhellg/more-feature-engineering-feature-selection-0-817?scriptVersionId=173223907&cellId=11).
* TFIDF vector [here](https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-cv-0-799-lb-0-799?scriptVersionId=172203959&cellId=16).
* TFIDF word [here](https://www.kaggle.com/code/guillaums/error-in-tfidf-vectorizer-in-baseline-nbs?scriptVersionId=175110986&cellId=17).

# 1. Import modules

In [ ]:
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

In [ ]:
from typing import List
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import warnings
import logging
import os
import shutil
import json, string
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import cohen_kappa_score, accuracy_score
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
from spellchecker import SpellChecker
import lightgbm as lgb

# logging setting 

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()

# 2. Load dataset and initial configuration

In [ ]:
class PATHS:
    train_path = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv'
    test_path = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv'
    sub_path = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv'

In [ ]:
class CFG:
    n_splits = 5
    seed = 42
    num_labels = 6

In [ ]:
train = pd.read_csv("/kaggle/input/persuade-2-0/AES2-persuade.csv")
# train = train[(train["src"] == "kaggle-only") | (train["src"] == "kaggle-persuade")]

train["fold"] = 1
train[(train["src"] == "kaggle-only") | (train["src"] == "kaggle-persuade")]["fold"] = 0

train = train[["essay_id", "full_text", "score", "prompt_label", "fold"]]
train.head()

In [ ]:
test = pd.read_csv(PATHS.test_path)
test.head(3)

# 3. Feature Engineering

## 3.1 Data preprocessing functions definations

In [ ]:
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)


cList = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have",
    "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
    "he'd": "he would",  ## --> he had or he would
    "he'd've": "he would have","he'll": "he will", "he'll've": "he will have", "he's": "he is", 
    "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is",
    "I'd": "I would",   ## --> I had or I would
    "I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not",
    "it'd": "it had",   ## --> It had or It would
    "it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
    "let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have",
    "must've": "must have","mustn't": "must not","mustn't've": "must not have",
    "needn't": "need not","needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not","oughtn't've": "ought not have",
    "shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
    "she'd": "she would",   ## --> It had or It would
    "she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
    "should've": "should have","shouldn't": "should not","shouldn't've": "should not have",
    "so've": "so have","so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have","that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have","there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have",
    "to've": "to have","wasn't": "was not","weren't": "were not",
    "we'd": "we had",
    "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
    "what'll": "what will","what'll've": "what will have","what're": "what are","what's": "what is","what've": "what have",
    "when's": "when is","when've": "when have",
    "where'd": "where did","where's": "where is","where've": "where have",
    "who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
    "will've": "will have","won't": "will not","won't've": "will not have",
    "would've": "would have","wouldn't": "would not","wouldn't've": "would not have",
    "y'all": "you all","y'alls": "you alls","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
    "y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have",
    "you're": "you are",  "you've": "you have"
}
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def dataPreprocessing(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Remove \xa0
    x = x.replace(u'\xa0',' ')
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
#     x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

def remove_punctuation(text):
    # string.punctuation
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def dataPreprocessing_w_contract_punct_remove(x):
    # Convert words to lowercase
    x = x.lower()
    # Remove HTML
    x = removeHTML(x)
    # Delete strings starting with @
    x = re.sub("@\w+", '',x)
    # Delete Numbers
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    # Delete URL
    x = re.sub("http\w+", '',x)
    # Replace consecutive empty spaces with a single space character
    x = re.sub(r"\s+", " ", x)
    x = expandContractions(x)
    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\.+", ".", x)
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    x = remove_punctuation(x)
    # Remove empty characters at the beginning and end
    x = x.strip()
    return x

## 3.2 Paragraph based feature

<a id='paragraph-feature'></a>

In [ ]:
# TODO: can be fixed by keeping "\n" and removed empty paragraph entries
columns = [(pl.col("full_text").str.split(by="\n\n").alias("paragraph"))]
train = pl.from_pandas(train).with_columns(columns)
test = pl.from_pandas(test).with_columns(columns)

In [ ]:
# paragraph features
def Paragraph_Preprocess(tmp):
    # Expand the paragraph list into several lines of data
    tmp = tmp.explode('paragraph')
    # Paragraph preprocessing
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(dataPreprocessing))
    # Calculate the length of each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len"))
    # Calculate the number of sentences and words in each paragraph
    tmp = tmp.with_columns(pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias("paragraph_sentence_cnt"),
                    pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias("paragraph_word_cnt"),)
    return tmp

# feature_eng
paragraph_fea = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']
def Paragraph_Eng(train_tmp):
    aggs = [
        # Count the number of paragraph lengths greater than and less than the i-value
        *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f"paragraph_{i}_cnt") for i in [50,75,100,125,150,175,200,250,300,350,400,500,600,700] ], 
        *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f"paragraph_{i}_cnt") for i in [25,49]], 
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in paragraph_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in paragraph_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in paragraph_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in paragraph_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in paragraph_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in paragraph_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in paragraph_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in paragraph_fea],  
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in paragraph_fea],
    ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Paragraph_Preprocess(train)
train_feats = Paragraph_Eng(tmp)

# Obtain feature names
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(5)

## 3.3 Sentence based features

source: https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments/notebook#Features-engineering

In [ ]:
# sentence feature
def Sentence_Preprocess(tmp):
    # Preprocess full_text and use periods to segment sentences in the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=".").alias("sentence"))
    tmp = tmp.explode('sentence')
    # Calculate the length of a sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x)).alias("sentence_len"))
    # Filter out the portion of data with a sentence length greater than 15
    tmp = tmp.filter(pl.col('sentence_len')>=15)
    # Count the number of words in each sentence
    tmp = tmp.with_columns(pl.col('sentence').map_elements(lambda x: len(x.split(' '))).alias("sentence_word_cnt"))
    return tmp

# feature_eng
sentence_fea = ['sentence_len','sentence_word_cnt']
def Sentence_Eng(train_tmp):
    aggs = [
        # Count the number of sentences with a length greater than i
        *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f"sentence_{i}_cnt") for i in [15,50,100,150,200,250,300] ], 
        # other
        *[pl.col(fea).max().alias(f"{fea}_max") for fea in sentence_fea],
        *[pl.col(fea).mean().alias(f"{fea}_mean") for fea in sentence_fea],
        *[pl.col(fea).min().alias(f"{fea}_min") for fea in sentence_fea],
        *[pl.col(fea).first().alias(f"{fea}_first") for fea in sentence_fea],
        *[pl.col(fea).last().alias(f"{fea}_last") for fea in sentence_fea],
        *[pl.col(fea).sum().alias(f"{fea}_sum") for fea in sentence_fea],
        *[pl.col(fea).kurtosis().alias(f"{fea}_kurtosis") for fea in sentence_fea],
        *[pl.col(fea).quantile(0.25).alias(f"{fea}_q1") for fea in sentence_fea], 
        *[pl.col(fea).quantile(0.75).alias(f"{fea}_q3") for fea in sentence_fea], 
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Sentence_Preprocess(train)

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

## 3.4 Word based feature

source: https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments/notebook#Features-engineering

In [ ]:
# word feature
def Word_Preprocess(tmp):
    # Preprocess full_text and use spaces to separate words from the text
    tmp = tmp.with_columns(pl.col('full_text').map_elements(dataPreprocessing).str.split(by=" ").alias("word"))
    tmp = tmp.explode('word')
    # Calculate the length of each word
    tmp = tmp.with_columns(pl.col('word').map_elements(lambda x: len(x)).alias("word_len"))
    # Delete data with a word length of 0
    tmp = tmp.filter(pl.col('word_len')!=0)
    
    return tmp

# feature_eng
def Word_Eng(train_tmp):
    aggs = [
        # Count the number of words with a length greater than i+1
        *[pl.col('word').filter(pl.col('word_len') >= i+1).count().alias(f"word_{i+1}_cnt") for i in range(15) ], 
        # other
        pl.col('word_len').max().alias(f"word_len_max"),
        pl.col('word_len').mean().alias(f"word_len_mean"),
        pl.col('word_len').std().alias(f"word_len_std"),
        pl.col('word_len').quantile(0.25).alias(f"word_len_q1"),
        pl.col('word_len').quantile(0.50).alias(f"word_len_q2"),
        pl.col('word_len').quantile(0.75).alias(f"word_len_q3"),
        ]
    df = train_tmp.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas()
    return df

tmp = Word_Preprocess(train)

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

## 3.5 Character TFIDF feature:

For TFIDF vector generation we use TfidfVectorizer provided by [sickit-learn liberay](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

#### Terms:
* **TF (Term frequency)**: Number of time a term occur in a document / Total number of term in the document.
* **DF (Document frequency)**: Number of document where the term appear / Total number of document.
* **IDF (Inverse Document Frequency)**: 1 / Document frequency
***
### TfidfVectorizer parameters:
* **tokenizer**: Is set to `lambda x: x` which means the text will be passed as it is. 
* **preprocessor**: Is set to `lambda x: x` which means the text will be passed as it is.
* **token_pattern**: Is not set to `None` means word will be taken as token as it is without any word-level processing.
* **strip_accents**: Ts set to `unicode` which means include unicode characters during preprocessing step.
* **analyzer**: Ts set to `word` which means the feature (terms or token) will be the words
* **ngram_range**: ngram_range equal to `(1, 2)` which means unigrams and bigrams
* **min_df**: Is equal to `0.05` means ignore terms that occur in less the 5% of documents.
* **max_df**: Is equal to `0.95` means ignore terms that occur in more them 95% of documents.
* **sublinear_tf**: Is equal to `True` means replace tf with 1 + log(tf)

##### Note:
* **tokenizer=lambda x: x**: "`words are not tokenized from full-text? Tokenizer should only be overided by identity if text is already tokenized before. Perhaps vectorizer is receiving string (char sequence) instead of word sequence, so it behaves like a char ngram vectorizer`" qouted from notebook [here](https://www.kaggle.com/code/guillaums/error-in-tfidf-vectorizer-in-baseline-nbs?scriptVersionId=175110986&cellId=11)

In [ ]:
# TfidfVectorizer parameter
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(1,3),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])

print("#"*80)
vect_feat_names=vectorizer.get_feature_names_out()
print(vect_feat_names[100:110])
print("#"*80, "\n\n")

# Convert to array
dense_matrix = train_tfid.toarray()

# Convert to dataframe
df = pd.DataFrame(dense_matrix)

# rename features
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = train_feats['essay_id']

# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

<a id='tfidf-word-feature'></a>
## 3.6 Word TFIDF feature:

Source notebook [here](https://www.kaggle.com/code/guillaums/error-in-tfidf-vectorizer-in-baseline-nbs?scriptVersionId=175110986&cellId=17)

In [ ]:
stopwords_list = stopwords.words('english')
# TfidfVectorizer parameter
word_vectorizer = TfidfVectorizer(
    strip_accents='ascii',
    analyzer = 'word',
    ngram_range=(1,1),
    min_df=0.15,
    max_df=0.85,
    sublinear_tf=True,
    stop_words=stopwords_list,
)
# Fit all datasets into TfidfVector,this may cause leakage and overly optimistic CV scores
processed_text = train.to_pandas()["full_text"].progress_apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))
train_tfid = word_vectorizer.fit_transform([i for i in processed_text])

# Convert to array
dense_matrix = train_tfid.toarray()
# Convert to dataframe
df = pd.DataFrame(dense_matrix)
# rename features
tfid_w_columns = [ f'tfid_w_{i}' for i in range(len(df.columns))]
df.columns = tfid_w_columns
df['essay_id'] = train_feats['essay_id']

df.head()
# Merge the newly generated feature data with the previously generated feature data
train_feats = train_feats.merge(df, on='essay_id', how='left')

feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

<a id='extra-feature'></a>
## 3.7 Extra features:
Reference: https://www.kaggle.com/code/tsunotsuno/updated-debertav3-lgbm-with-spell-autocorrect

In [ ]:
class Preprocessor:
    def __init__(self) -> None:
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        self.spellchecker = SpellChecker()

    def spelling(self, text):
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))
        return amount_miss
    
    def count_sym(self, text, sym):
        sym_count = 0
        for l in text:
            if l == sym:
                sym_count += 1
        return sym_count

    def run(self, data: pd.DataFrame, mode:str) -> pd.DataFrame:
        
        # preprocessing the text
        data["processed_text"] = data["full_text"].apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))
        
        # Text tokenization
        data["text_tokens"] = data["processed_text"].apply(lambda x: word_tokenize(x))
        
        # essay length
        data["text_length"] = data["processed_text"].apply(lambda x: len(x))
        
        # essay word count
        data["word_count"] = data["text_tokens"].apply(lambda x: len(x))
        
        # essay unique word count
        data["unique_word_count"] = data["text_tokens"].apply(lambda x: len(set(x)))
        
        # essay sentence count
        data["sentence_count"] = data["full_text"].apply(lambda x: len(x.split('.')))
        
        # essay paragraph count
        data["paragraph_count"] = data["full_text"].apply(lambda x: len(x.split('\n\n')))
        
        # count misspelling
        data["splling_err_num"] = data["processed_text"].progress_apply(self.spelling)
        print("Spelling mistake count done")
        
        # ratio fullstop / text_length ** new
        data["fullstop_ratio"] = data["full_text"].apply(lambda x: x.count(".")/len(x))
        
        # ratio comma / text_length ** new
        data["comma_ratio"] = data["full_text"].apply(lambda x: x.count(",")/len(x))
        
        return data
    
preprocessor = Preprocessor()
tmp = preprocessor.run(train.to_pandas(), mode="train")
train_feats = train_feats.merge(tmp, on='essay_id', how='left')
feature_names = list(filter(lambda x: x not in ['essay_id','score'], train_feats.columns))
print('Features Number: ',len(feature_names))
train_feats.head(3)

## 3.8 Test dataset featurization

In [ ]:
# Paragraph
tmp = Paragraph_Preprocess(test)
test_feats = Paragraph_Eng(tmp)

# Sentence
tmp = Sentence_Preprocess(test)
test_feats = test_feats.merge(Sentence_Eng(tmp), on='essay_id', how='left')

# Word
tmp = Word_Preprocess(test)
test_feats = test_feats.merge(Word_Eng(tmp), on='essay_id', how='left')

# Tfidf
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfid_{i}' for i in range(len(df.columns))]
df.columns = tfid_columns
df['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df, on='essay_id', how='left')

# Word Tfidf
processed_text = test.to_pandas()["full_text"].progress_apply(lambda x: dataPreprocessing_w_contract_punct_remove(x))
test_w_tfid = word_vectorizer.transform([i for i in processed_text])
dense_matrix = test_w_tfid.toarray()
df_w = pd.DataFrame(dense_matrix)
tfid_w_columns = [ f'tfid_w_{i}' for i in range(len(df_w.columns))]
df_w.columns = tfid_w_columns
df_w['essay_id'] = test_feats['essay_id']
test_feats = test_feats.merge(df_w, on='essay_id', how='left')

# Extra feature
tmp = preprocessor.run(test.to_pandas(), mode="train")
test_feats = test_feats.merge(tmp, on='essay_id', how='left')

# Features number
feature_names = list(filter(lambda x: x not in ['essay_id','score'], test_feats.columns))
print('Features number: ',len(feature_names))
test_feats.head(3)

# 4. Data preparation

## 4.1 Add k-fold details

In [ ]:
# skf = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
# for i, (_, val_index) in enumerate(skf.split(train_feats, train_feats["score"], train_feats["prompt_label"])):
#     train_feats.loc[val_index, "fold"] = i
# print(train_feats.shape)
# train_feats.head()

## 4.2 Feature selection

In [ ]:
target = "score"
train_drop_columns = ["essay_id", "fold", "full_text", "paragraph", "text_tokens", "processed_text", "prompt_label"] + [target]

In [ ]:
train_feats.drop(columns=train_drop_columns).head()

In [ ]:
test_drop_columns = ["essay_id", "full_text", "paragraph", "text_tokens", "processed_text"]

In [ ]:
test_feats.drop(columns=test_drop_columns).head()

# 5. Training

## 5.1 Evaluation function and loss function defination 

In [ ]:
# idea from https://www.kaggle.com/code/rsakata/optimize-qwk-by-lgb/notebook#QWK-objective
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = (y_true + a).round()
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True

def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

def qwk_param_calc(y):
    a = y.mean()
    b = (y ** 2).mean() - a**2
    return np.round(a, 4), np.round(b, 4)
# a = 2.948
# b = 1.092

## 5.2 Training LGBMRegressor model

In [ ]:
models = []

callbacks = [
    lgb.log_evaluation(period=25), 
    lgb.early_stopping(stopping_rounds=75,first_metric_only=True)
]
for fold in range(CFG.n_splits):

    model = lgb.LGBMRegressor(
        objective = qwk_obj, metrics = 'None', learning_rate = 0.1, max_depth = 5,
        num_leaves = 10, colsample_bytree=0.5, reg_alpha = 0.1, reg_lambda = 0.8,
        n_estimators=1024, random_state=CFG.seed, verbosity = - 1
    )
    
    a, b = qwk_param_calc(train_feats[train_feats["fold"] != fold]["score"])
    
    # Take out the training and validation sets for 5 kfold segmentation separately
    X_train = train_feats[train_feats["fold"] != fold].drop(columns=train_drop_columns)
    y_train = train_feats[train_feats["fold"] != fold]["score"] - a

    X_eval = train_feats[train_feats["fold"] == fold].drop(columns=train_drop_columns)
    y_eval = train_feats[train_feats["fold"] == fold]["score"] - a

    print('\nFold_{} Training ================================\n'.format(fold+1))
    print(f"Fold {fold} a: {a}  ;;  b: {b}")
    # Training model
    lgb_model = model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_eval, y_eval)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks
    )
    models.append(model)

## 5.3 Validating LGBMRegressor model

In [ ]:
preds, trues = [], []
    
for fold, model in enumerate(models):
    X_eval_cv = train_feats[train_feats["fold"] == fold].drop(columns=train_drop_columns)
    y_eval_cv = train_feats[train_feats["fold"] == fold]["score"]

    pred = model.predict(X_eval_cv) + a
    
    trues.extend(y_eval_cv)
    preds.extend(np.round(pred, 0))

v_score = cohen_kappa_score(trues, preds, weights="quadratic")

print(f"Validation score : {v_score}")

## 5.4 Testing and collecting prediction

In [ ]:
# predecting for 5 models
preds = []
for fold, model in enumerate(models):
    X_eval_cv = test_feats.drop(columns=test_drop_columns)
    # pred = model.predict(X_eval_cv)
    pred = model.predict(X_eval_cv) + a
    preds.append(pred)

# Combining the 5 model results
for i, pred in enumerate(preds):
    test_feats[f"score_pred_{i}"] = pred
test_feats["score"] = np.round(test_feats[[f"score_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1),0).astype('int32')

In [ ]:
test_feats.head()

# 6. Submission

In [ ]:
test_feats[["essay_id", "score"]].to_csv("submission.csv", index=False)